<a href="https://colab.research.google.com/github/efaro2014/Upper_vs_Lower/blob/main/Keras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
from google.colab import auth
auth.authenticate_user()
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'Efrem'
!gcloud config set project {project_id}

ERROR: (gcloud.config.set) The project property must be set to a valid project ID, not the project name [Efrem]
To set your project, run:

  $ gcloud config set project PROJECT_ID

or to unset it, run:

  $ gcloud config unset project


In [ ]:
! mkdir /tmp/metadata

mkdir: cannot create directory ‘/tmp/metadata’: File exists


In [ ]:
! mkdir -p /tmp/features

In [ ]:
# load metadata
!gsutil cp gs://virgosvs-ml-train/upper-vs-lower/metadata/* /tmp/metadata

Copying gs://virgosvs-ml-train/upper-vs-lower/metadata/training_metadata.csv...
/ [1 files][ 42.5 KiB/ 42.5 KiB]                                                
Operation completed over 1 objects/42.5 KiB.                                     


In [ ]:
! /tmp/features/*

/bin/bash: /tmp/features/110777.mp4.hdf5: Permission denied


In [ ]:
# load features
!gsutil cp gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/*.mp4.hdf5 /tmp/features

Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/110777.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/112066.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/112072.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/112078.mp4.hdf5...
- [4 files][678.4 MiB/678.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/112082.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/120286.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-vs-lower/resnet50-features/padded/120287.mp4.hdf5...
Copying gs://virgosvs-ml-train/upper-

In [ ]:
import numpy as np
import pandas as pd
import h5py

In [ ]:
metadata = pd.read_csv('/tmp/metadata/training_metadata.csv').query('available_to_intern == True')
metadata = metadata[metadata.clip_id != 164662]

In [ ]:
ids = metadata.clip_id.values

In [ ]:
metadata.loc[metadata.institution == "CDC", "test_set"] = 1
metadata["test_set"] = metadata["test_set"].fillna(0)

In [ ]:
metadata['test_set']

0      0.0
2      0.0
3      0.0
5      0.0
6      0.0
      ... 
147    0.0
148    0.0
149    0.0
151    0.0
152    0.0
Name: test_set, Length: 129, dtype: float64

In [ ]:
train_clip_ids = metadata.query('test_set==0').clip_id.values # train clip ids


In [ ]:
test_clip_ids = metadata.loc[metadata.test_set == 1].clip_id.values

In [ ]:
test_clip_ids

array([164547, 164548, 164549, 164556, 164557, 164559, 164563, 164565,
       164568, 164569, 164572, 164573, 164584, 164587, 164596, 164598,
       164599, 164600, 164606, 164610])

In [ ]:
# metadata['labels'] = metadata.labels.replace({'upper':0., 'lower': 1.})
metadata['labels'] = metadata['labels'].replace({'upper':0, 'lower':1}) # convert label to 1=lower 0=upper

In [ ]:
from keras.utils.np_utils import to_categorical
labels = to_categorical(metadata.labels)

In [ ]:
# labels

In [ ]:
def get_feature_path(clip_id):
    return "/tmp/features/{}.mp4.hdf5".format(clip_id)

def get_dataset(clip_id, hdf5_file):
    '''
    grab the 'dataset' component of the hdf5_file.
    '''
    return hdf5_file["tmp/features/{}.mp4.hdf5".format(clip_id)]

In [ ]:
train_hdf5_files = [h5py.File(get_feature_path(clip_id),'r') for clip_id in train_clip_ids]
test_hdf5_files = [h5py.File(get_feature_path(clip_id),'r') for clip_id in test_clip_ids]




In [ ]:
hdf5_files = [h5py.File(get_feature_path(clip_id),'r') for clip_id in ids]

In [ ]:
len(train_hdf5_files), len(test_hdf5_files)


(109, 20)

In [ ]:
set(train_hdf5_files).intersection(set(test_hdf5_files)) # check dataleakage --> empty set


set()

In [ ]:
trainset = {} # {'clip_id': dataset, '':}
for clip_id, hdf5_file in zip(train_clip_ids, train_hdf5_files):
    trainset[clip_id] = get_dataset(clip_id, hdf5_file)

testset = {}
for clip_id, hdf5_file in zip(test_clip_ids, test_hdf5_files):
    testset[clip_id] = get_dataset(clip_id, hdf5_file)

In [ ]:
trainset[train_clip_ids[0]][0].shape

(2048,)

In [ ]:
 dataset_labels = dict(zip(metadata['clip_id'], labels)) # {clip_id: label}

In [ ]:
# dataset_labels

In [ ]:
clip_id = train_clip_ids[0]
frame_data = trainset[clip_id]
random_idxs = np.random.choice(len(frame_data), size=20)
random_idxs.sort()
random_frames = frame_data[random_idxs]
random_frames


array([[0.78480613, 0.18085735, 0.55615497, ..., 0.30356652, 0.20505346,
        1.206269  ],
       [0.43852425, 0.15506268, 0.01746236, ..., 0.        , 0.24620907,
        0.14649667],
       [1.9493349 , 0.6037125 , 0.16683033, ..., 0.11855523, 0.26602083,
        0.16930553],
       ...,
       [1.9735545 , 0.154144  , 0.00463026, ..., 0.01139247, 1.0278682 ,
        0.13544235],
       [0.35198265, 0.02602563, 0.        , ..., 0.        , 0.024617  ,
        0.14581598],
       [0.49227694, 0.8163984 , 0.03303427, ..., 0.01302283, 0.27322417,
        0.05570493]], dtype=float32)

In [ ]:
def get_random_frames(clip_id, n=1):
    frame_data = trainset[clip_id] # only trainset
    random_idxs = np.random.choice(len(frame_data), size=n)
    random_idxs.sort()
    return frame_data[random_idxs]

def random_batch(clip_list, batch_size=32):
    x = np.zeros(shape=(batch_size, 2048))
    y = np.zeros(shape=(batch_size,2))
    for i in range(batch_size):
        clip_id = np.random.choice(clip_list)
        frames = get_random_frames(clip_id) # 1 frame if n=1
        clip_label = dataset_labels[clip_id]
        x[i] = frames
        y[i] = clip_label
    return x, y

def gen_test_batch(test_clip_ids,batch_size=32):
    #test_frames = np.sum([testset[clip_id].shape[0] for clip_id in test_clip_ids])
    # x = np.zeros(shape=(batch_size, 2048))
    x = np.vstack([testset[clip_id] for clip_id in test_clip_ids])
    # y = np.vstack([[dataset_labels[clip_id]] * testset[clip_id].shape[0] for clip_id in test_clip_ids]).reshape(-1, 1)
    y = np.vstack([[dataset_labels[clip_id]] * testset[clip_id].shape[0] for clip_id in test_clip_ids]).reshape(-1, 2)
    return x, y


In [ ]:
# gen big training batch
batch_x, batch_y = random_batch(train_clip_ids, 3000)   
print('X shape:', batch_x.shape, 'y shape:', batch_y.shape)

X shape: (3000, 2048) y shape: (3000, 2)


In [ ]:
np.unique(batch_y, return_counts=True) # y_train distribution

(array([0., 1.]), array([3000, 3000]))

In [ ]:
x_test, y_test = gen_test_batch(test_clip_ids)   
print('X shape:', x_test.shape, 'y shape:', y_test.shape)

X shape: (440827, 2048) y shape: (440827, 2)


In [ ]:
print(y_test.shape)

(440827, 2)


In [ ]:
np.unique(y_test, return_counts=True) # y_test distribution

(array([0., 1.], dtype=float32), array([440827, 440827]))

In [ ]:
# Model Trainig with Keras 

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping


In [ ]:
feature_dim = 2048
model = Sequential()

model.add(Dense(50, activation='relu', input_shape=(feature_dim,)))
model.add(Dense(2, activation='softmax'))
model.summary() 

# compile
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=[keras.metrics.AUC()])
#
earlystopping = EarlyStopping(patience=5, restore_best_weights=True)
# train !
history = model.fit(batch_x, 
                    batch_y,
                    epochs=100, # Number of passes over complete dataset
                    verbose=True, 
                    batch_size = 64,
                    callbacks=[earlystopping],
                    validation_split=0.2)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 50)                102450    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 102,552
Trainable params: 102,552
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
38/38 [==============================] - 2s 16ms/step - loss: 0.8969 - auc_1: 0.5241 - val_loss: 0.5766 - val_auc_1: 0.7807
Epoch 2/100
38/38 [==============================] - 0s 7ms/step - loss: 0.5274 - auc_1: 0.8242 - val_loss: 0.4344 - val_auc_1: 0.8882
Epoch 3/100
38/38 [==============================] - 0s 6ms/step - loss: 0.4365 - auc_1: 0.8819 - val_loss: 0.3924 - val_auc_1: 0.9044
Epoch 4/100
38/38 [==============================] - 0s 7ms/step - loss: 0.3635 - auc_1: 0.9203 - val_lo

In [ ]:
# Evaluate
loss, auc = model.evaluate(x_test, 
                          y_test,
                          verbose=True)
print(f"Test loss: {loss:.3}")
print(f"Test AUC: {auc:.3%}")

13776/13776 [==============================] - 27s 2ms/step - loss: 0.5558 - auc_1: 0.8426
Test loss: 0.556
Test AUC: 84.264%


In [ ]:
def predict_video_class(clip_ids):
    avg_pred = []
    for clip_id in clip_ids:
        pred = model.predict(np.array(testset[clip_id]))
        avg = np.mean(pred, axis=0)
        avg_pred.append(avg)  
    return np.array(avg_pred)

In [ ]:
cdc_preds = predict_video_class(test_clip_ids)


In [ ]:
cdc_labels =[dataset_labels[clip_id] for clip_id in test_clip_ids]
cdc_labels

[array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32),
 array([1., 0.], dtype=float32),
 array([1., 0.], dtype=float32)]

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, f1_score

In [ ]:
# test_result = pd.DataFrame(data={'clip_id': test_clip_ids,
#                    'labels': cdc_labels,
#                    'predict_proba': cdc_preds})
# # test_result

In [ ]:
y_score = np.array(cdc_preds).reshape(-1,1)
y_score


array([[0.35217988],
       [0.6478236 ],
       [0.83675563],
       [0.16324967],
       [0.845216  ],
       [0.15478382],
       [0.28559253],
       [0.7144089 ],
       [0.288456  ],
       [0.7115467 ],
       [0.7972085 ],
       [0.20279072],
       [0.27283862],
       [0.7271632 ],
       [0.24688275],
       [0.7531177 ],
       [0.7882585 ],
       [0.2117432 ],
       [0.7457228 ],
       [0.25427702],
       [0.8665539 ],
       [0.13344595],
       [0.76391506],
       [0.23608421],
       [0.26655936],
       [0.733445  ],
       [0.27184287],
       [0.7281621 ],
       [0.8528165 ],
       [0.14718406],
       [0.8062099 ],
       [0.19379571],
       [0.319133  ],
       [0.68086845],
       [0.32191914],
       [0.6780825 ],
       [0.76836777],
       [0.23163413],
       [0.7953548 ],
       [0.20464812]], dtype=float32)

In [ ]:
# roc_auc_score(cdc_labels, np.array(cdc_preds).reshape(-1,1))
roc_auc_score(cdc_labels, cdc_preds)

1.0

HyperParameter Tunning 
Randomized Search 

In [ ]:
def create_model(learning_rate=3e-3, reg_coeff=0.001,nl=1,nn=256):
    model = keras.models.Sequential()
    model.add(Dense(50, activation='relu', input_shape=(feature_dim,)))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(units=1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(reg_coeff)) )
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    # compile
      
    model.compile(loss='categorical_crossentropy', 
            optimizer=optimizer,
            metrics=[keras.metrics.AUC()])
    return model

feature_dim = 2048
# model = Sequential()



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Import sklearn wrapper from keras
from keras.wrappers.scikit_learn import KerasClassifier
# Create a model as a sklearn 
estimatormodel = KerasClassifier(create_model)

keras_clf = keras.wrappers.scikit_learn.KerasClassifier(create_model, epochs=4, batch_size=64)
params = {
    "reg_coeff": [0.001, 0.01], 
    "learning_rate": [0.001,0.01]
    # "nl": [1, 2, 9],
    # "nn":[10,50,100,1000]
}

rnd_search_cv = RandomizedSearchCV(keras_clf, params, n_iter=1, cv=3, scoring='roc_auc')
rnd_search_cv.fit(batch_x, batch_y)

Epoch 1/4
313/313 [==============================] - 2s 4ms/step - loss: 0.6505 - auc_67: 0.6777
Epoch 2/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4961 - auc_67: 0.8432
Epoch 3/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4493 - auc_67: 0.8751
Epoch 4/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4201 - auc_67: 0.8927


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/4
313/313 [==============================] - 2s 3ms/step - loss: 0.6443 - auc_68: 0.7150
Epoch 2/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4759 - auc_68: 0.8563
Epoch 3/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4370 - auc_68: 0.8818
Epoch 4/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4009 - auc_68: 0.9025


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/4
313/313 [==============================] - 1s 3ms/step - loss: 0.6227 - auc_69: 0.7170
Epoch 2/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4908 - auc_69: 0.8470
Epoch 3/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4429 - auc_69: 0.8791
Epoch 4/4
313/313 [==============================] - 1s 3ms/step - loss: 0.4135 - auc_69: 0.8964


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/4
469/469 [==============================] - 2s 3ms/step - loss: 0.6153 - auc_70: 0.7217
Epoch 2/4
469/469 [==============================] - 1s 3ms/step - loss: 0.4671 - auc_70: 0.8634
Epoch 3/4
469/469 [==============================] - 1s 3ms/step - loss: 0.4203 - auc_70: 0.8926
Epoch 4/4
469/469 [==============================] - 1s 3ms/step - loss: 0.3923 - auc_70: 0.9068


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f729f5a7a10>,
                   iid='deprecated', n_iter=1, n_jobs=None,
                   param_distributions={'learning_rate': [0.001, 0.01],
                                        'reg_coeff': [0.001, 0.01]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [ ]:
print("Best: {} using {}".format(rnd_search_cv.best_score_,rnd_search_cv.best_params_))

Best: 0.90288710533033 using {'reg_coeff': 0.001, 'learning_rate': 0.001}


Keras Tuner

In [ ]:
! mkdir /tmp/test_videos

In [ ]:
!gsutil cp gs://virgosvs-ml-train/upper-vs-lower/additional/clips/*.mp4 /tmp/test_videos

Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152003.224378.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152034.224418.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152039.224426.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152040.224427.mp4...
/ [4 files][  1.1 GiB/  1.1 GiB]   58.9 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152043.224431.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152049.224439.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152052.224443.mp4...
Copying gs://virgosvs-ml-train/upper-vs-lower/additional/clips/152053.224444.mp

In [ ]:
import cv2
from keras.applications.resnet50 import ResNet50
model = ResNet50( weights='imagenet', pooling='avg', include_top = False) # avg pooling
GREY_THRESH = 5
PAD_COLOR = [0, 0, 0]
def _nongrey_pixel_mask(frame, grey_threshold=GREY_THRESH):
    greyscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    color_diff = cv2.absdiff(frame, cv2.cvtColor(greyscale, cv2.COLOR_GRAY2BGR))
    color_diff_grey = cv2.cvtColor(color_diff, cv2.COLOR_BGR2GRAY)
    _, non_grey_pixels = cv2.threshold(color_diff_grey, grey_threshold, 255, cv2.THRESH_BINARY)
    return non_grey_pixels


def _get_bounding_box(frame, grey_threshold=GREY_THRESH):
    nongrey_pixels = _nongrey_pixel_mask(frame, grey_threshold)
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(5, 5))
    tmp = cv2.cvtColor(nongrey_pixels, cv2.COLOR_GRAY2BGR)
    tmp = cv2.morphologyEx(tmp, cv2.MORPH_OPEN, kernel, iterations=3)
    tmp = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    return cv2.boundingRect(tmp)


def get_median_bounding_box(path, n_samples=20, grey_threshold=GREY_THRESH):
    video = cv2.VideoCapture(path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_frames = np.linspace(0, total_frames - 1, num=n_samples).astype(int)

    bbs = []
    for frame_idx in sample_frames:
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = video.read()
        bb = _get_bounding_box(frame, grey_threshold=grey_threshold)
        bbs.append(bb)

    return np.median(bbs, axis=0)

In [ ]:
class FramePreprocessor:
    def __init__(self, target_w: int, target_h: int):
        self.target_w = target_w
        self.target_h = target_h
        self.target_ratio = target_w/float(target_h)

    def simple_crop(self, frame, bounding_box):
        # crop:
        left, right = int(bounding_box[0]), int(bounding_box[0] + bounding_box[2])
        top, bottom = int(bounding_box[1]), int(bounding_box[1] + bounding_box[3])
        cropped_frame = frame[top:bottom, left:right]
        # resize:
        input_w, input_h = bounding_box[2], bounding_box[3]
        input_ratio = input_w / float(input_h)
        if input_ratio > self.target_ratio:
            resized_w = self.target_w
            resized_h = int(self.target_w/input_ratio)
        else:
            resized_h = self.target_h
            resized_w = int(self.target_h*input_ratio)
        resized = cv2.resize(cropped_frame, (resized_w, resized_h))
        # pad:
        delta_w, delta_h = self.target_w - resized_w, self.target_h - resized_h
        top_pad, bottom_pad = delta_h // 2, delta_h - (delta_h // 2)
        left_pad, right_pad = delta_w // 2, delta_w - (delta_w // 2)
        padded = cv2.copyMakeBorder(resized, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT,
                                    value=PAD_COLOR)
        return padded

In [ ]:
from keras.utils import Sequence
import cv2
import numpy as np
pretrained_mean = [103.939, 116.779, 123.68]
class VideoSequence(Sequence):
    def __init__(self, video_path, batch_size, w=224, h=224):
        self.video_path = video_path
        self.batch_size = batch_size
        self.position = 0
        self.video = cv2.VideoCapture(video_path)
        self.total_frames = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        if w is None:
            w = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.w = w
        if h is None:
            h = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.h = h
        self.bounding_box = get_median_bounding_box(video_path)
        self.fpp = FramePreprocessor(self.w, self.h)
    def __len__(self):
        return int(np.ceil(self.total_frames / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_stop = (idx + 1) * self.batch_size
        #handling for the last batch
        batch_size = min(batch_stop, self.total_frames) - batch_start
        # if we can't sequentially read, we need to seek.
        if batch_start != self.position:
            self.video.set(cv2.CAP_PROP_POS_FRAMES, batch_start)
            self.position = batch_start
         
        frame_batch = np.zeros(shape=(batch_size, self.h, self.w, 3), dtype=np.uint8)
        for i in range(0, batch_size):
            _, frame = self.video.read()
            frame = self.fpp.simple_crop(frame, self.bounding_box)
            frame = frame.astype('float64')
            for k in range(3):
                frame[:,:,k] -= pretrained_mean[k]
            frame_batch[i] = frame
            self.position += 1
        return frame_batch

In [ ]:
import os
def filelist(root):
    """Return a fully-qualified list of filenames under root directory; sort names alphabetically."""
    allfiles = []
    for path, subdirs, files in os.walk(root):
        for name in files:
          if name.endswith('.mp4'):
            allfiles.append(os.path.join(path, name))
          else:
            continue
    return sorted(allfiles)
  
file_names = filelist('/tmp/test_videos/')

file_names[5]

'/tmp/test_videos/152049.224439.mp4'

In [ ]:
# def predict_raw_video_class(file_names):
#     predictions = []
#     for file in file_names:
#         video = VideoSequence(file, 32)
#         feature = model.predict(video, batch_size=32)
#         pred = np.mean(model.predict(feature), axis=0)
#         predictions.append(pred)  
#     return np.array(predictions)


In [ ]:
! mkdir '/tmp/new_features/'

mkdir: cannot create directory ‘/tmp/new_features/’: File exists


In [ ]:
import h5py
import re

In [ ]:
files = file_names[6:8] # list of video path , '/tmp/164316.mp4', '/tmp/164329.mp4'

def gen_feature_files(model, files, prefix='/tmp/new_features/', batch_size=32):
  '''
  save features (.hdf5 format) from ResNet50 into the specified file path
  parameters
  ---------------
  model: Resnet50 model instance
  prefix: file path prefix
  files: list of video path
  Example ->  ['/tmp/110777.mp4', '/tmp/164316.mp4'] 


  '''
  for video_path in files:
    clip_id = re.search("(\d+).mp4$", video_path).group(1)
    video = VideoSequence(video_path, batch_size)
    feature = model.predict(video, batch_size=batch_size)
    file_name = f'{prefix}{clip_id}.mp4.hdf5'
    with h5py.File(file_name, 'w') as hf:
        hf.create_dataset(file_name,  data=feature)
        print(f'Save {clip_id}.mp4 features to {file_name}')

In [ ]:
# gen_feature_files(model, files)

In [ ]:
# predict_raw_video_class(file_names[0:1])




In [ ]:
def predict_raw_video_class(file_names, l2_coeff=0.001):
    predictions = []
    for file in file_names:
        print(f"Inferencing video {file}")
        video = VideoSequence(file, 32)
        feature = model.predict(video, batch_size=32)
        pred = np.mean(models[l2_coeff].predict(feature))
        predictions.append(pred)  
    return np.array(predictions)

In [ ]:
def predict_video_class(clip_ids):
    avg_pred = []
    for clip_id in clip_ids:
        pred = model.predict(np.array(testset[clip_id]))
        avg = np.mean(pred, axis=0)
        avg_pred.append(avg)  
    return np.array(avg_pred)
cdc_preds = predict_video_class(test_clip_ids)


In [ ]:
import argparse
from tensorflow.keras.datasets import mnist

from ray.tune.integration.keras import TuneReportCallback

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing")
args, _ = parser.parse_known_args()


def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    import tensorflow as tf
    batch_size = 128
    num_classes = 10
    epochs = 12

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config["hidden"], activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.SGD(
            lr=config["lr"], momentum=config["momentum"]),
        metrics=["accuracy"])

    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({
            "mean_accuracy": "accuracy"
        })])



    import ray
    from ray import tune
    from ray.tune.schedulers import AsyncHyperBandScheduler
    mnist.load_data()  # we do this on the driver because it's not threadsafe
    ray.shutdown()
    ray.init(num_cpus=4 if args.smoke_test else None)
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20)

    analysis = tune.run(
        train_mnist,
        name="exp",
        scheduler=sched,
        metric="mean_accuracy",
        mode="max",
        stop={
            "mean_accuracy": 0.99,
            "training_iteration": 5 if args.smoke_test else 300
        },
        num_samples=10,
        resources_per_trial={
            "cpu": 2,
            "gpu": 0
        },
        config={
            "threads": 2,
            "lr": tune.uniform(0.001, 0.1),
            "momentum": tune.uniform(0.1, 0.9),
            "hidden": tune.randint(32, 512),
        })
    print("Best hyperparameters found were: ", analysis.best_config)


2021-03-29 20:58:18,254	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc,hidden,lr,momentum
train_mnist_7f14d_00000,RUNNING,,214,0.0352817,0.787862


(pid=4761) 2021-03-29 20:58:22.577894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=4761) 2021-03-29 20:58:22.577894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
(pid=4761) 2021-03-29 20:58:25.462310: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=4761) 2021-03-29 20:58:25.462310: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=4761) 2021-03-29 20:58:25.496393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(pid=4761) 2021-03-29 20:58:25.496393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(pid=4761) 2021-03-29 20:58:25.564782: E tensorflow/stream_executor/cuda/cuda_driver.cc:

Result for train_mnist_7f14d_00000:
  date: 2021-03-29_20-58-29
  done: false
  experiment_id: 4320c1ba1ac84ae598b5cb4d273b32cb
  hostname: 82eedec80e15
  iterations_since_restore: 1
  mean_accuracy: 0.8762166500091553
  node_ip: 172.28.0.2
  pid: 4761
  time_since_restore: 5.341347694396973
  time_this_iter_s: 5.341347694396973
  time_total_s: 5.341347694396973
  timestamp: 1617051509
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7f14d_00000
  


Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_7f14d_00000,RUNNING,172.28.0.2:4761,214,0.0352817,0.787862,0.876217,1,5.34135
train_mnist_7f14d_00001,PENDING,,122,0.0865463,0.686742,,,


...

Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_7f14d_00009,RUNNING,172.28.0.2:5443,166,0.0984966,0.674162,0.981567,10,23.4329
train_mnist_7f14d_00000,TERMINATED,,214,0.0352817,0.787862,0.980483,12,30.5803
train_mnist_7f14d_00001,TERMINATED,,122,0.0865463,0.686742,0.979483,12,22.844
train_mnist_7f14d_00002,TERMINATED,,481,0.012584,0.278352,0.937917,12,47.2769
train_mnist_7f14d_00003,TERMINATED,,349,0.0969487,0.880783,0.990017,9,29.691
train_mnist_7f14d_00004,TERMINATED,,490,0.0562747,0.18449,0.971917,12,47.7958
train_mnist_7f14d_00005,TERMINATED,,370,0.0641523,0.336615,0.975517,12,40.1471
train_mnist_7f14d_00006,TERMINATED,,490,0.0740172,0.266777,0.977767,12,49.097
train_mnist_7f14d_00007,TERMINATED,,52,0.0991786,0.799609,0.965033,12,17.4495
train_mnist_7f14d_00008,TERMINATED,,57,0.0473355,0.613397,0.959417,12,17.4294


Result for train_mnist_7f14d_00009:
  date: 2021-03-29_21-04-19
  done: true
  experiment_id: f85ba8ee8ae040b68f61055e251516e0
  experiment_tag: 9_hidden=166,lr=0.098497,momentum=0.67416
  hostname: 82eedec80e15
  iterations_since_restore: 12
  mean_accuracy: 0.9837499856948853
  node_ip: 172.28.0.2
  pid: 5443
  time_since_restore: 27.607826232910156
  time_this_iter_s: 2.0735113620758057
  time_total_s: 27.607826232910156
  timestamp: 1617051859
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: 7f14d_00009
  


Trial name,status,loc,hidden,lr,momentum,acc,iter,total time (s)
train_mnist_7f14d_00000,TERMINATED,,214,0.0352817,0.787862,0.980483,12,30.5803
train_mnist_7f14d_00001,TERMINATED,,122,0.0865463,0.686742,0.979483,12,22.844
train_mnist_7f14d_00002,TERMINATED,,481,0.012584,0.278352,0.937917,12,47.2769
train_mnist_7f14d_00003,TERMINATED,,349,0.0969487,0.880783,0.990017,9,29.691
train_mnist_7f14d_00004,TERMINATED,,490,0.0562747,0.18449,0.971917,12,47.7958
train_mnist_7f14d_00005,TERMINATED,,370,0.0641523,0.336615,0.975517,12,40.1471
train_mnist_7f14d_00006,TERMINATED,,490,0.0740172,0.266777,0.977767,12,49.097
train_mnist_7f14d_00007,TERMINATED,,52,0.0991786,0.799609,0.965033,12,17.4495
train_mnist_7f14d_00008,TERMINATED,,57,0.0473355,0.613397,0.959417,12,17.4294
train_mnist_7f14d_00009,TERMINATED,,166,0.0984966,0.674162,0.98375,12,27.6078


2021-03-29 21:04:19,153	INFO tune.py:450 -- Total run time: 357.21 seconds (357.17 seconds for the tuning loop).


Best hyperparameters found were:  {'threads': 2, 'lr': 0.09694874184722062, 'momentum': 0.8807830251886724, 'hidden': 349}
